In [14]:

import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
import tensorflow as tf

from torch.autograd import grad
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from pdefind import *
from PDE_Equation import pde_matrix_mul, sparse_coeff, normalized_xi_threshold, pde_Recover


In [15]:
# Prepare dataset
data = sio.loadmat(os.path.join(os.getcwd(), "data", "Advection_diffusion.mat"))
usol = np.real(data['Expression1'])
usol= usol.reshape((51,51,61,4))

x = usol[:,:,:,0]
y = usol[:,:,:,1]
t = usol[:,:,:,2]
u = usol[:,:,:,3]

X = np.transpose((t.flatten(),x.flatten(), y.flatten()))
y = u.reshape((u.size, 1))
print(X.shape, y.shape)
print('X:', X)

(158661, 3) (158661, 1)
X: [[ 3.  -5.  -5. ]
 [ 3.1 -5.  -5. ]
 [ 3.2 -5.  -5. ]
 ...
 [ 8.8  5.   5. ]
 [ 8.9  5.   5. ]
 [ 9.   5.   5. ]]


In [16]:
# Add noise
noise_level = 0.1
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

In [17]:
#number_of_samples = 2000
#idx = np.random.permutation(y.size)
#X_train = X[idx, :][:number_of_samples]
#y_train = y_noisy[idx, :][:number_of_samples]
#print('X_train', X_train.shape)
#print('y_train', y_train.shape)

idxs = np.random.choice(y.size, 2000, replace=False)

X_train = torch.tensor(X[idxs], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y_noisy[idxs], dtype=torch.float32)
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
#print("X shape", X.shape)
#print("y shape", y.shape)
#print('X_train', X_train)

X_train shape torch.Size([2000, 3])
y_train shape torch.Size([2000, 1])


In [19]:
polynm = ['1', 'u']
spa_der = ['1', 'u_{x}', 'u_{y}','u_{xx}', 'u_{yy}','u_{xy}']
library_coeffs = pde_matrix_mul(polynm, spa_der)
print('library_coeffs:', library_coeffs)

tot_items = len(library_coeffs)
print('tot_items:', tot_items)

library_coeffs: ['1', 'u_{x}', 'u_{y}', 'u_{xx}', 'u_{yy}', 'u_{xy}', 'u', 'uu_{x}', 'uu_{y}', 'uu_{xx}', 'uu_{yy}', 'uu_{xy}']
tot_items: 12


In [20]:
# Setup Network
net = PINN(sizes=[3,20,15,10,5,1], activation=torch.nn.Tanh())
print(net)

PINN(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=15, bias=True)
    (3): Tanh()
    (4): Linear(in_features=15, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=5, bias=True)
    (7): Tanh()
    (8): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [21]:
#X_train = torch.from_numpy(X_train)
print(type(X_train))
uhat = net(X_train)
print(X_train)

<class 'torch.Tensor'>
tensor([[ 7.1000,  4.2000, -1.2000],
        [ 5.5000,  4.0000, -4.6000],
        [ 6.2000,  2.0000, -4.6000],
        ...,
        [ 8.0000, -2.6000, -3.6000],
        [ 8.7000,  2.6000,  3.6000],
        [ 3.3000,  1.6000, -1.8000]], requires_grad=True)


In [81]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        print('poly.shape', poly.shape)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    dudt = du[:, 0:1]
    dudx = du[:, 1:2]
    dudy = du[:, 2:3]
    dudu = grad(outputs=dudx, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    dudxx = dudu[:, 1:2]
    dudxy = dudu[:, 2:3]
    dudyy = grad(outputs=dudy, inputs=data, 
              grad_outputs=torch.ones_like(dudxx), create_graph=True)[0]
   
    #dudu = torch.cat((torch.ones_like(dudt), du[:,0:1]), dim=1)
    dudu = torch.cat((torch.ones_like(dudx), dudx, dudy, dudxx, dudyy[:, 2:3], dudxy), dim=1)
    #for order in np.arange(2, library_config['deriv_order']+1):
    #    du = tf.concat((du, tf.gradients(du[:, order-1], data)[0][:, 0:1]), axis=1)
    
    #for o in np.arange(1, deriv_order):
        #du2 = grad(outputs=dudu[:,o:o+1], inputs=data, 
                  #grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
       # dudx = torch.cat((dudx, du2[:,0:1]), dim=1)


    
    # build all possible combinations of poly and dudx vectors
    theta = None
    for i in range(poly_order+1):
        #print('i:', i)
        for j in range(5+1):
            #print('j:', j)
            comb = poly[:,i:i+1] * dudu[:,j:j+1]
            
            if theta is None:
                theta = comb
            else:
                theta = torch.cat((theta, comb), dim=1)
                
    return dudu, theta

In [82]:
uhat = net(X_train)
dudu, theta = build_library_2D(X_train, uhat, poly_order=1, deriv_order=2)
#print(dudu.shape)
print('theta:', theta.shape)
print('dudu.shape:', dudu.shape)

poly.shape torch.Size([2000, 2])
theta: torch.Size([2000, 12])
dudu.shape: torch.Size([2000, 6])


In [40]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        print('poly.shape:', poly.shape)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    return du
  

In [41]:
uhat = net(X_train)
du = build_library_2D(X_train, uhat, poly_order=1, deriv_order=2)
print(du)

poly.shape: torch.Size([2000, 2])
tensor([[-0.0177,  0.0255, -0.0112],
        [-0.0169,  0.0155, -0.0049],
        [-0.0133,  0.0201, -0.0073],
        ...,
        [-0.0025,  0.0029,  0.0014],
        [-0.0008,  0.0097, -0.0053],
        [-0.0281,  0.0375, -0.0153]], grad_fn=<MmBackward>)


In [28]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    u_t = du[:, 0:1]
    u_x = du[:, 1:2]
    u_y = du[:, 2:3]
    du2 = tf.gradients(u_x,data)[0]
    u_xx = du2[:, 1:2]
    u_xy = du2[:, 2:3]
    u_yy = grad(outputs=u_y[:, 2:3], inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    #tf.gradients(u_y,data)[0][:, 2:3]
    du = torch.cat((torch.ones_like(u_x), u_x, u_y , u_xx, u_yy, u_xy), dim=1)
    
    return du
  

In [29]:
uhat = net(X_train)
du = build_library_2D(X_train, uhat, poly_order=2, deriv_order=2)
print(du)

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [83]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    dudt = du[:, 0:1]
    dudx = du[:, 1:2]
    dudy = du[:, 2:3]
    dudu = grad(outputs=dudx, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    dudxx = dudu[:, 1:2]
    dudxy = dudu[:, 2:3]
    dudyy = grad(outputs=dudy[:, 2:3], inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
   
    dudu = torch.cat((torch.ones_like(dudt), du[:,0:1]), dim=1)
   # torch.cat((torch.ones_like(dudx), dudx, dudy, dudxx, dudyy, dudxy), dim=1)
    #for order in np.arange(2, library_config['deriv_order']+1):
    #    du = tf.concat((du, tf.gradients(du[:, order-1], data)[0][:, 0:1]), axis=1)
    
    for o in np.arange(1, deriv_order):
        du2 = grad(outputs=dudu[:,o:o+1], inputs=data, 
                  grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
        dudx = torch.cat((dudx, du2[:,0:1]), dim=1)


    
    # build all possible combinations of poly and dudx vectors
    theta = None
    for i in range(poly_order+1):
        for j in range(deriv_order+1):
            comb = poly[:,i:i+1] * dudu[:,j:j+1]
            
            if theta is None:
                theta = comb
            else:
                theta = torch.cat((theta, comb), dim=1)
                
    return dudt, theta

In [ ]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    dudt = du[:, 0:1]
    dudx = du[:, 1:2]
    dudy = du[:, 2:3]
    dudu = grad(outputs=dudx, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    dudxx = dudu[:, 1:2]
    dudxy = dudu[:, 2:3]
    dudyy = grad(outputs=dudy, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
   
    #dudu = torch.cat((torch.ones_like(dudt), du[:,0:1]), dim=1)
    dudu = torch.cat((torch.ones_like(dudx), dudx, dudy, dudxx, dudyy, dudxy), dim=1)
    #for order in np.arange(2, library_config['deriv_order']+1):
    #    du = tf.concat((du, tf.gradients(du[:, order-1], data)[0][:, 0:1]), axis=1)
    
    for o in np.arange(1, deriv_order):
        du2 = grad(outputs=dudu[:,o:o+1], inputs=data, 
                  grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    dudu = torch.cat((dudu, du2[:,0:1]), dim=1)


    
    # build all possible combinations of poly and dudx vectors
    theta = None
    for i in range(poly_order+1):
        for j in range(deriv_order+1):
            comb = poly[:,i:i+1] * dudu[:,j:j+1]
            
            if theta is None:
                theta = comb
            else:
                theta = torch.cat((theta, comb), dim=1)
                
    return dudt, theta